In [ ]:
'''
 Continue Watching
 Tiếp tục xem – Hiển thị danh sách anime đang xem dở, sắp xếp theo thời gian xem gần nhất.
Input:
user_id
Lịch sử xem (anime_id, watched_episodes, watching_status)
Output:
Danh sách anime chưa xem hết (anime_id, progress_percentage, last_watched_episode)
'''

In [91]:
import pyodbc
import pandas as pd
import os
from sqlalchemy import create_engine

In [ ]:
from dotenv import load_dotenv

load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
engine = create_engine(
    f"mssql+pyodbc://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}?driver=ODBC+Driver+17+for+SQL+Server"
)

In [ ]:
def continue_watch(user_id: int):
    q = 'SELECT * FROM ANIMELISTD WHERE USER_ID = :user_id'
    df = pd.read_sql(q, engine, params={"user_id": user_id})
    q2 = 'SELECT * FROM anime_refined_modified'
    df_anime = pd.read_sql(q2,engine)
    
    #Join lấy thông tin anime
    watching = df.merge(df_anime, on='anime_id', how='left')[['user_id', 'anime_id','watching_status','watched_episodes', 'Name', 'rating','Genres', 'year','Producers']].dropna(subset=['Name'])
    
    #Lấy các anime đang xem và sắp xếp theo số tập đã xem giảm dần
    df_top5 = watching[watching['watching_status']==1].sort_values(['user_id', 'watched_episodes'], ascending=[True, False]).drop_duplicates()
    
    #lấy 5 anime đã được xem nhiều nhất của user
    recently_watch = df_top5.groupby('user_id').head(5)[['user_id', 'anime_id', 'Name', 'rating','Genres', 'year','Producers']]
    
    #Trả về dictionary chứa thông tin anime
    dict_result = recently_watch[recently_watch['user_id']==user_id].set_index('anime_id').to_dict(orient='index')
    return dict_result

In [ ]:
'''
def continue_watch1(user_id: int):
    df = pd.read_csv('animelist.csv')
    df_anime = pd.read_csv('anime_refined_modified.csv')
    #Join lấy thông tin anime
    watching = df.merge(df_anime, on='anime_id', how='left')[['user_id', 'anime_id','watching_status','watched_episodes', 'Name', 'rating','Genres', 'year','Producers']].dropna(subset=['Name'])
    
    #Lấy các anime đang xem và sắp xếp theo số tập đã xem giảm dần
    df_top5 = watching[watching['watching_status']==1].sort_values(['user_id', 'watched_episodes'], ascending=[True, False]).drop_duplicates()
    
    #lấy 5 anime đã được xem nhiều nhất của user
    recently_watch = df_top5.groupby('user_id').head(5)[['user_id', 'anime_id', 'Name', 'rating','Genres', 'year','Producers']]
    
    #Trả về dictionary chứa thông tin anime
    dict_result = recently_watch[recently_watch['user_id']==user_id].set_index('anime_id').to_dict(orient='index')
    return dict_result'
'''